In [365]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.vector_ar.vecm import VECMResults
from patsy import dmatrices
import itertools

In [366]:
class PairsTrade:
    '''Class for Backtesting a Pairs Trading strategy using the cointegration method of pair creation with speed of adjustment filtering'''

    def __init__(self, start, end, formation_period, trading_period): # Constructor Method
        
        self.start = start
        self.end = end
        self.formation_period = slice(*formation_period)
        self.trading_period = slice(*trading_period)
        self.get_data()
        self.get_close_prices()
        self.clean_data()  
        self.intorder()
        self.normalize_price_series()
        self.cointegration_testing()


    def get_data(self):

        self.table = pd.read_html('https://en.wikipedia.org/wiki/FTSE_100_Index')[4]
        self.tickers_raw = self.table.iloc[:,1]
        self.tickers_Series = self.tickers_raw + ".L"
        self.tickers = self.tickers_Series.tolist()
        self.data = yf.download(self.tickers, start = self.start, end = self.end)
        if self.data.empty:
            print(f'No Data pulled for period {self.start} : {self.end}')

        else:
            swapped_columnindex = self.data.columns.swaplevel(0,1)
            data_copy = self.data.copy()
            data_copy.columns = swapped_columnindex
            data_copy = data_copy.sort_index(axis = 1, level = 0)
            self.data = data_copy
            self.data.columns = self.data.columns.set_levels(self.data.columns.levels[0].str.replace('.L', "", regex = False), level = 0)
            print("Data successfully retrieved")
            print('=' * 55)

    def plot_stock(self,symbol, cols=None):
        self.symbol = symbol
        if cols is None:
            cols = "Close"
        self.data.xs(symbol, level = 0, axis = 1)[cols].plot(figsize = (10,6), title = (f'A Graph of {self.symbol} {cols} Prices'))

    def get_close_prices(self):
        self.df = self.data.xs('Close', level=1, axis=1)

    
    def get_date_price(self, symbol , bar):
        rawdate = pd.to_datetime(bar)
        if rawdate in self.df.index:
            price = self.df.loc[rawdate, symbol]
            return str(rawdate.date()), price
        else:
            print(f"Date {rawdate} not found in data.")

    def clean_data(self):
        initial_column_count = self.df.shape[1]
        self.df_cleaned = self.df.dropna(axis=1, how = 'any')
        dropped_columns_count = initial_column_count - self.df_cleaned.shape[1]
        print(f"Number of columns dropped during cleaning for stationarity testing: {dropped_columns_count}")
        total_nans_cleaned = self.df_cleaned.isna().sum().sum()
        print(f"Total Number of NANs in the entire DataFrame after column removal: {total_nans_cleaned}")
        self.df = self.df_cleaned

    # Order of Integration Testing

    def intorder(self):
        def adf_test(series):
            result = sm.tsa.stattools.adfuller(series, autolag = 'BIC')
            return result[1]
        self.adfp = self.df.apply(adf_test, axis = 0) > 0.05
        diff_series = self.df.loc[:, self.adfp].diff().dropna()
        self.adfp2 = diff_series.apply(adf_test, axis = 0) > 0.05
        adfp2_alligned = self.adfp2.reindex(self.df.columns, fill_value = True)
        self.i1_prices = self.df.loc[:, ~adfp2_alligned]
        print(f" {len(self.i1_prices.columns)} I(1) Stocks")
        print("=" * 55)


    def normalize_price_series(self):
        self.norm_i1prices = self.i1_prices/ self.i1_prices.iloc[0]

    # Cointegration testing

    def cointegration_testing(self):
        self.formation_prices = self.norm_i1prices[self.formation_period]
        self.trading_prices = self.norm_i1prices[self.trading_period]

        self.stock_combinations = list(itertools.combinations(self.norm_i1prices.columns, 2))
        self.cointegrated_pairs = []
        self.cointegration_results = {
            'Stock 1': [],
            'Stock 2': [],
            'Alpha': [],
            'Beta': []
        }
        for stock1 , stock2 in self.stock_combinations:

            Pairs_close = self.formation_prices[[stock1, stock2]]
            Pairs_close = Pairs_close.asfreq('B').ffill()

            #lag selection 
            var_model = sm.tsa.VAR(Pairs_close)
            lag_selection = var_model.select_order(maxlags = 15, trend = 'c')
            optimal_lag = lag_selection.bic
            self.var_result = var_model.fit(optimal_lag)

            vecm = sm.tsa.VECM(Pairs_close)
            
            # Serial autocorrelation test - issue need to return
            #self.residuals = self.var_result.resid
            #out = sm.stats.stattools.durbin_watson(var_model.resid)
            #print(out) 

            coint_johansen_result = coint_johansen(Pairs_close, 0, optimal_lag)
            vecm_results = VECMResults(Pairs_close, deterministic='ci')
            trace_stat = coint_johansen_result.lr1
            trace_critical_value = coint_johansen_result.cvt
           
            if trace_stat[0] > trace_critical_value[1,2]:
                print(f"For Stock Pair {stock1}, {stock2}:")
                print(f"Trace Statistic: {trace_stat}")
                print(f"Trace Statistic Critical Values (10%, 5%, 1%):\n{trace_critical_value}\n")
                print(f"Stock Pair {stock1}, {stock2} is cointegrated")
                print("=" * 55)
                self.cointegrated_pairs.append((stock1, stock2))

                alpha = vecm_results.alpha
                beta = vecm_results.beta

                self.cointegration_results['Stock 1'].append(stock1)
                self.cointegration_results['Stock 2'].append(stock2)
                self.cointegration_results['Alpha'].append(alpha)
                self.cointegration_results['Beta'].append(beta)

        self.cointegration_df = pd.DataFrame(self.cointegration_results)
        self.cointegrated_pairs = self.cointegration_df.iloc[:, :2]
        
        print(f"Out of {len(self.stock_combinations)}  possible pairs there are {len(self.cointegration_df)} cointegrated pairs") #n.b very high percentage of coiintegrated pairs












            









        
        



        

        



        


In [367]:
self = PairsTrade(start = "2022-01-01", end = "2023-12-25", formation_period=("2022-01-01", "2023-01-01"), trading_period=("2023-01-01", "2023-12-25"))

[*********************100%%**********************]  100 of 100 completed


Data successfully retrieved
Number of columns dropped during cleaning for stationarity testing: 16
Total Number of NANs in the entire DataFrame after column removal: 0
 64 I(1) Stocks


TypeError: VECMResults.__init__() missing 8 required positional arguments: 'exog', 'exog_coint', 'k_ar', 'coint_rank', 'alpha', 'beta', 'gamma', and 'sigma_u'

In [361]:
view = self.cointegration_results
pairs = self.cointegrated_pairs

In [360]:
# obtain speed of adjustment coefficients and filter based on speed
